By: Dominikus Krisna Herlambang | ©2021

## Sentinel-2

Sentinel-2 at a glance:
1. Sentinel-2 is a European optical imaging mission consisting of two constellation satellites, Sentinel-2A and Sentinel-2B, orbiting the poles in a sun-synchronous orbit at an altitude of 786 km. These two satellites are 180 degrees apart from each other and are medium resolution satellites with a temporal resolution of 10 days for one satellite or 5 days with two satellites.

2. Sentinel-2 has a multispectral imager with 13 spectral bands which can be used for operational observations such as land cover maps, land change detection maps, and geophysical variables. The coverage area of ​​each image is 290 km.

3. The data generated by Sentinel-2 is called Sentinel-2 L1C Data and has been processed to Level-1C (ToA Reflectance). This product has been UTM/WGS84 ortho projected and uses a Digital Elevation Model (DEM) projected on cartographic coordinates.

For this Live Coding, we will focus on Sentinel-2 data which is managed directly by COPERNICUS which is available in the Google Earth Catalog.

Therefore, we need to integrate or authenticate first between Google Colaboratory and Google Earth Engine!

# The Code

### Package Installation

In [1]:
!pip install geemap

In [2]:
# Import Package
import ee
import geemap

# Authenticate Earth Engine account
#ee.Authenticate()
#ee.Initialize()

In [3]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

### PART 1

In [4]:
# Set target location
point = ee.Geometry.Point(104.66914633458155, -3.25663744146642038);

In [5]:
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)

In [11]:
# Call up the Sentinel 2 image collection by searching for Sentinel in the search bar
dataset = (
    ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
    .filterDate('2022-01-01', '2022-01-31')
    # Pre-filter to get less cloudy granules.
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    .map(mask_s2_clouds)
)

### PART 2

In [12]:
# For the actual color composite, band_4=Red, band_3=Green, band_2=blue
rgb_vis = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

In [13]:
# Navigate to the area to be researched
# Set target location
m = geemap.Map()
m.set_center(104.66914633458155, -3.2566374414664203, 9)

# Add true color images to the map console
m.add_layer(dataset.median(), rgb_vis, 'RGB')
m

Map(center=[-3.2566374414664203, 104.66914633458155], controls=(WidgetControl(options=['position', 'transparen…